In [688]:
# Import necessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import (
    PowerTransformer,
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
)
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pickle
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import Ridge

from scipy import stats
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.pipeline import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import re
from sklearn.impute import SimpleImputer

from ydata_profiling import ProfileReport
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score

In [689]:
# Load data
excel_file_path = "./train.csv"
df = pd.read_csv(excel_file_path, encoding="latin-1")

In [690]:
df = df[(df['Listening_Time_minutes'] != 0)]
df['Listening_Time_minutes'] = np.log1p(df['Listening_Time_minutes'])

In [691]:
def gen_eda():
    profile = ProfileReport(
        pd.concat([df], axis=1),
        title="Pandas Profiling Report",
        explorative=True,
    )
    profile.to_file("pandas_profiling_report.html")


# gen_eda()

In [692]:
df.sample(5)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
381030,381030,Fashion Forward,Episode 8,106.42,Lifestyle,62.85,Monday,Night,0.65,0.0,Negative,3.996938
656144,656144,Money Matters,Episode 62,81.79,Business,80.43,Wednesday,Evening,49.53,0.0,Negative,4.257373
704976,704976,Game Day,Episode 78,68.27,Sports,55.81,Tuesday,Afternoon,61.37,1.0,Positive,3.561739
519036,519036,Melody Mix,Episode 81,116.21,Music,66.03,Saturday,Evening,13.15,1.0,Positive,4.710678
144367,144367,Digital Digest,Episode 43,117.29,Technology,60.32,Tuesday,Evening,13.71,1.0,Negative,4.747438


In [693]:
df.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,741449.000000,655066.000000,741449.000000,598546.000000,741448.000000,741449.000000
mean,374957.717282,65.185957,59.836673,52.328988,1.348355,3.619211
std,216507.054088,32.568739,22.874244,28.493094,1.152670,0.777568
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000560
25%,187430.000000,36.730000,39.380000,28.340000,0.000000,3.213869
50%,374926.000000,64.420000,60.020000,53.780000,1.000000,3.802549
75%,562461.000000,94.330000,79.500000,76.760000,2.000000,4.190771
max,749999.000000,325.240000,119.460000,119.910000,103.910000,4.795543


In [694]:
df = df.dropna(subset=[
    "Episode_Length_minutes",
    # "Guest_Popularity_percentage"
])
print(df["Episode_Length_minutes"].isnull().sum(), df["Guest_Popularity_percentage"].isnull().sum())

0 120984


In [695]:
def remove_outliers(df, outlier_dict):
    for distribution, category in outlier_dict.items():
        if distribution == "normal":
            for cat in category:
                upper_limit = df[cat].mean() + 3 * df[cat].std()
                lower_limit = df[cat].mean() - 3 * df[cat].std()
                print(cat, upper_limit, lower_limit)
                # capping
                # df[cat] = np.where(df[cat] > upper_limit,upper_limit,np.where(df[cat] < lower_limit, lower_limit, df[cat]))
                # Trimming
                df = df[(df[cat] < upper_limit) & (df[cat] > lower_limit)]
        elif distribution == "skew":
            for cat in category:
                percentile25 = df[cat].quantile(0.4)
                percentile75 = df[cat].quantile(0.75)
                iqr = percentile75 - percentile25
                upper_limit = percentile75 + 1.5 * iqr
                lower_limit = percentile25 - 1.5 * iqr
                print(cat, upper_limit, lower_limit)
                # capping
                # df[cat] = np.where(
                #     df[cat] > upper_limit,
                #     upper_limit,
                #     np.where(df[cat] < lower_limit, lower_limit, df[cat]),
                # )
                # Trimming
                df = df[(df[cat] < upper_limit) & (df[cat] > lower_limit)]
    return df

In [696]:
outlier_dict = {
    "normal": [],
    "skew": [],
}


def pre_process(df):
    df["Publication_Day"] = (
        df["Publication_Day"]
        .map(
            {
                "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7,
            }
        )
        .fillna(0)
        .astype(int)
    )
    df["Publication_Time"] = (
        df["Publication_Time"]
        .map({"Morning": 1, "Afternoon": 2, "Evening": 3, "Night": 4})
        .fillna(0)
        .astype(int)
    )
    df["Episode_Sentiment"] = (
        df["Episode_Sentiment"]
        .map({"Negative": 0, "Neutral": 1, "Positive": 2})
        .fillna(0)
        .astype(int)
    )
    return df


df = pre_process(df)
df = remove_outliers(df, outlier_dict)

In [697]:
df.to_csv("df.csv", index=False)

In [698]:
# Define features and target
def get_X_Y(df):
    X = df.drop(columns=["id", "Listening_Time_minutes", "Episode_Title"])
    Y = df["Listening_Time_minutes"]
    return X, Y


X, Y = get_X_Y(df)
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=5
)
print(X_train.shape)

(524052, 9)


In [699]:
# Get the list of categorical column names
categories_order = {
    "Publication_Day": sorted(list(df["Publication_Day"].unique())),
    "Publication_Time": sorted(list(df["Publication_Time"].unique())),
    "Episode_Sentiment": sorted(list(df["Episode_Sentiment"].unique())),
}
categorical_feat_ord = list(categories_order.keys())
categorical_feat_nom = [ "Podcast_Name", "Genre"]
categorical = categorical_feat_nom + categorical_feat_ord
numerical_features = [col for col in X_train.columns if col not in categorical]

In [700]:
# Separate transformers for categorical and numerical features

# trf = FunctionTransformer(np.log1p, validate=True)
# trf = PowerTransformer()
# trf = FunctionTransformer(np.sqrt, validate=True)
# trf = FunctionTransformer(np.sin)
trf = StandardScaler()
# trf = MinMaxScaler()
trf = FunctionTransformer(np.log1p, validate=True)
# Add Polynomial Features
poly = PolynomialFeatures(degree=2, include_bias=False)

numerical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), 
           ("poly", poly),
           ("log", trf)]
)
categorical_transformer_onehot = Pipeline(
    steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)
categorical_transformer_ordinal = Pipeline(
    steps=[
        ("ord", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ]
)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer_onehot, categorical_feat_nom),
        ("cat_1", categorical_transformer_ordinal, categorical_feat_ord),
        ("num", numerical_transformer, numerical_features),
    ]
)

from sklearn.linear_model import Lasso, LinearRegression

model = LinearRegression()
model = Ridge(alpha=0.1)
# model = XGBRegressor(objective='reg:squarederror', scale_pos_weight=1)

pipeline = Pipeline([("preprocessor", preprocessor), ("model", model)])

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

              Podcast_Name  Episode_Length_minutes       Genre  \
472403      Market Masters                   42.25    Business   
125853         Funny Folks                  102.39      Comedy   
96601     Lifestyle Lounge                   65.30   Lifestyle   
577975           Humor Hub                    6.70      Comedy   
668138     Business Briefs                   47.30    Business   
...                    ...                     ...         ...   
639062         Sound Waves                   55.68       Music   
142615         Tech Trends                  116.82  Technology   
623682       Fitness First                   98.28      Health   
21346     Lifestyle Lounge                   90.38   Lifestyle   
341072  True Crime Stories                   81.89  True Crime   

        Host_Popularity_percentage  Publication_Day  Publication_Time  \
472403                       41.22                1                 1   
125853                       60.21                6          

In [702]:
# # Calculate the correlation matrix
# correlation_matrix = df.corr()

# # Save the correlation matrix to a CSV file
# correlation_matrix.to_csv('correlation_matrix.csv', index=True)

In [703]:
# Save the fitted pipeline as a .pkl file
filename_pkl = "model.pkl"
pickle.dump(pipeline, open(filename_pkl, "wb"))
print(f"Model saved as {filename_pkl}")

# Evaluate the model
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")


def adjusted_r2_score(y_true, y_pred, X):
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)  # number of samples
    k = X.shape[1]  # number of features
    adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - k - 1))
    return r2, adj_r2


# Example usage:
r2, adj_r2 = adjusted_r2_score(Y_test, y_pred, X_test)
print(f"R² Score: {r2:.4f}")
print(f"Adjusted R² Score: {adj_r2:.4f}")

# scores = cross_val_score(model, X_train, Y_train, cv=5)
# print("Cross-validation scores:", scores)
# print("Mean score:", scores.mean())

# Define the columns expected by the model
column_names = X_train.columns


def generate_submission(test_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(test_file)
    df = pd.DataFrame(df)
    # Replace empty strings with NaN
    df.replace("", np.nan, inplace=True)
    df = pre_process(df)
    # Select the relevant columns
    filtered_df = df[column_names]
    predictions = pipeline.predict(filtered_df)
    # Load the original test file to keep the PassengerId column
    original_df = pd.read_csv(test_file)
    original_df["Listening_Time_minutes"] = predictions
    original_df["Listening_Time_minutes"] = np.expm1(
        original_df["Listening_Time_minutes"]
    )
    # Save the results to a new CSV file
    submission_df = original_df[["id", "Listening_Time_minutes"]]
    submission_df.to_csv("submission.csv", index=False)
    print("Submission file saved as 'submission.csv'")


# Generate the submission
test_file = "test.csv"
generate_submission(test_file)

Model saved as model.pkl
Mean Squared Error: 0.08005768085204351
Root Mean Squared Error: 0.2829446604056057
R² Score: 0.8688
Adjusted R² Score: 0.8688
Submission file saved as 'submission.csv'
